# Temporary file, print everything currently

In [1]:
from data_analysis import DataAnalysis
import os

## 1. basic analysis

In [17]:
def read_data2(filename):
    """
    helper method to read the file line by line
    :param filename: the file to read data from - csv
    :return: the complete data in a list
    """
    complete_data = []
    # open the file with utf8 encoding, split it at the comma (it should be csv)
    with open(filename, encoding="utf8") as f:
        for lines in f:
            # the train file is separates by semicolons instead of commas
            line = lines.strip("\n").split(',')
            if "pap" in filename:
                temp_line=[]
                if line[1] == "implausible":
                    temp_line.append('0')
                else:
                    temp_line.append('1')
                temp_line.append(line[0])
                complete_data.append(temp_line)
            else:
                complete_data.append(line)

    return complete_data[1:]  # skip the header


In [18]:
class DataAnalysis2(DataAnalysis):
    def __init__(self, filename):
         self.file_content = read_data2(filename)

def get_data(name,classnum_name,file_name):
    '''
    param name: name of the dataset, pep-3k or pap
    param classnum_name: for pap dataset, binary or multiclass
    param file_name: dev, test, train
    return: the complete data in a list
    '''
    current_dir =os.path.realpath(".")
    parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir, os.pardir))
    return DataAnalysis2(os.path.abspath(os.path.join(parent_dir,name,'train-dev-test-split',classnum_name,file_name+'.csv')))



In [19]:
def temp_print_dataset(filename:str): #like 'pap_binary_train'
    paras=filename.split('_')
    print(paras)
    pep_set=get_data(paras[0],paras[1],paras[2])
    print(pep_set.file_content)
    print(pep_set.dataset_statistics())
    print(pep_set.extract_word_tokens())
    print(pep_set.store_words())
    print(pep_set.total_word_count())
    print(pep_set.unique_word_count())
    print(pep_set.plot_word_frequency(10))
    print(pep_set.average_word_length())
    print(pep_set.pos_tags())
    print(pep_set.readability())
    print(pep_set.avg_readability())
    print(pep_set.plausibility_rating_count())
    print(pep_set.count_pos_bigrams())
    print(pep_set.plot_pos_dict(filename+'plot_pos_dict'))

In [49]:
temp_print_dataset('pap_binary_train')

In [ ]:
temp_print_dataset('pap_binary_dev')

In [ ]:
temp_print_dataset('pep-3k__train')

In [ ]:
temp_print_dataset('pep-3k__dev')

In [16]:
pep_set=get_data('pep-3k','','train')
print(pep_set.file_content)

[['1', 'dog shake cat'], ['1', 'student climb ship'], ['1', 'pipe vent vapor'], ['0', 'chef burn air'], ['0', 'boat crush air'], ['1', 'knife sharpen glass'], ['1', 'screwdriver fix door'], ['1', 'stick hook purse'], ['0', 'elephant shake smoke'], ['1', 'woman cook turkey'], ['1', 'mayor jail person'], ['1', 'man film shirt'], ['1', 'man crush child'], ['0', 'cat lick cloud'], ['0', 'crab move gorilla'], ['0', 'leaf attach rope'], ['1', 'oven cook steak'], ['1', 'dog squash grape'], ['1', 'elephant pull house'], ['1', 'woman erect pen'], ['0', 'water poison rock'], ['0', 'bottle ejaculate book'], ['1', 'man grab apple'], ['1', 'dog close door'], ['1', 'dog pull rope'], ['0', 'rabbit clench lion'], ['1', 'policeman bend lamp'], ['1', 'oil dampen trash'], ['0', 'wall hook fish'], ['0', 'air warm log'], ['0', 'oil lengthen brick'], ['0', 'wind scratch arm'], ['1', 'ant bite man'], ['1', 'neck hang necklace'], ['0', 'policeman halt oil'], ['0', 'kangaroo kick oil'], ['1', 'policeman cook d

In [15]:
pap_set=get_data('pap','binary','train')
print(pap_set.file_content)

[['1', 'event occurs year'], ['1', 'tortoise brings limb'], ['0', 'headliner overpowers function'], ['1', 'county receives hour'], ['0', 'traveler acknowledges recognition'], ['0', 'galaxy excludes power'], ['0', 'airport skips role'], ['1', 'fence secures castle'], ['1', 'son mentions friend'], ['0', 'broadcast concentrates alignment'], ['0', 'building postdates connotation'], ['0', 'presence impresses rearrangement'], ['0', 'breach accelerates seat'], ['0', 'fraternity endures researcher'], ['1', 'album breaks genre'], ['1', 'man leaves hometown'], ['1', 'population follows religion'], ['0', 'director enriches excellence'], ['1', 'nominee carries district'], ['1', 'government establishes relation'], ['1', 'restriction poses challenge'], ['1', 'lack leads productivity'], ['0', 'bazaar covers costume'], ['0', 'campus persuades scenery'], ['0', 'fork finds triumph'], ['1', 'team recovers medallion'], ['1', 'reduction mitigates condition'], ['0', 'communism holds rent'], ['1', 'band resc